In [1]:
import os
import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from random import shuffle
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression

patht="D://datasets//MURA//train//XR_ELBOW//True";
pathf="D://datasets//MURA//train//XR_ELBOW//False";
path_tt="D://datasets//MURA//valid//XR_ELBOW//False";
path_tf="D://datasets//MURA//valid//XR_ELBOW//False";

# creating labels for one hot encoding

def create_label(image_name):
    if image_name=='True':
        return np.array([1,0]);
    else:
        return np.array([0,1]);    

#resizing images

def create_train_image():
    training_data=[];
    for img1 in tqdm(os.listdir(patht)):
        path=os.path.join(patht,img1);
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        training_data.append([np.array(img_data),create_label('True')])
    for img1 in tqdm(os.listdir(pathf)):
        path=os.path.join(pathf,img1)
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        training_data.append([np.array(img_data),create_label('False')]);
    shuffle(training_data)
    np.save('train_data.npy',training_data);
    return training_data

#Createing test dataset

def create_test_data():
    testing_data=[];img_num=0
    for img in tqdm(os.listdir(path_tt)):
        path=os.path.join(path_tt,img);
        img_num=img_num+1;
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        testing_data.append([np.array(img_data),img_num])
    for img in tqdm(os.listdir(path_tf)):
        path=os.path.join(path_tf,img);
        img_num=img_num+1;
        img_data=cv2.imread(path,cv2.IMREAD_GRAYSCALE);
        img_data=cv2.resize(img_data,(50,50));
        testing_data.append([np.array(img_data),img_num])
    shuffle(testing_data)
    return testing_data

train_data=create_train_image();
train=train_data[:-800];
test=train_data[-800:];
X_train=np.array([i[0] for i in train]).reshape(-1,50,50,1)
y_train=np.array([i[1] for i in train])
X_test=np.array([i[0] for i in test]).reshape(-1,50,50,1);
y_test=[i[1] for i in test]

# Creating layers

def conv32(convnet,steps):
    for i in range(steps):
        convnet=conv_2d(convnet,32,5,activation='relu')
        convnet=max_pool_2d(convnet,5)#67
    return convnet;

def conv64(convnet,steps):
    for i in range(steps):
        convnet=conv_2d(convnet,64,5,activation='relu')
        convnet=max_pool_2d(convnet,5)#67
    return convnet;

def conv128(convnet,steps):
    for i in range(steps):
        convnet=conv_2d(convnet,128,5,activation='relu')
        convnet=max_pool_2d(convnet,5)#67
    return convnet;

def conv256(convnet,steps):
    for i in range(steps):
        convnet=conv_2d(convnet,256,5,activation='relu')
        convnet=max_pool_2d(convnet,5)#67
    return convnet;

convnet=input_data(shape=[None,50,50,1],name='input')
'''convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)#1
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)#2
convnet=conv_2d(convnet,128,5,activation='relu')
convnet=max_pool_2d(convnet,5)'''
convnet=conv32(convnet,1)
convnet=conv64(convnet,1)


convnet=conv128(convnet,1)
convnet=conv256(convnet,1)
convnet=conv128(convnet,1)
convnet=conv64(convnet,1)
convnet=conv32(convnet,1)
'''convnet=conv_2d(convnet,128,5,activation='relu')
convnet=max_pool_2d(convnet,5)#68
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)#69
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)#70'''
convnet=fully_connected(convnet,1024,activation='relu')
convnet=dropout(convnet,0.8)
convnet=fully_connected(convnet,2,activation='softmax')
convnet=regression(convnet,optimizer='adam',learning_rate=0.001,loss='categorical_crossentropy',name='complete')
model=tflearn.DNN(convnet,tensorboard_dir='log')
model.fit(X_train,y_train,n_epoch=50,validation_set=(X_test,y_test),snapshot_step=500,show_metric=True,run_id='Elbow')

'''test_data=create_test_data();'''
test_data=[]
'''for num,data in enumerate(test_data[:16]):
    img_num=data[1]
    img_data=data[0]
    orig=img_data
    data=img_data.reshape(50,50,1)
    model_out=model.predict([data])[0]
    if np.argmax(model_out)==1:
        str_label='Fracture'
    else:
        str_label='Not Fracture'
    plt.imshow(orig,cmap='gray')
    plt.title(str_label)
    plt.gca().set_axis_off()
    plt.show()'''
img_data=cv2.imread("D://datasets//MURA//train//XR_ELBOW//False//img3313.png",cv2.IMREAD_GRAYSCALE);
img_data=cv2.resize(img_data,(50,50));
test_data.append([np.array(img_data),1])
test_data.append([np.array(img_data),2])
for num,data in enumerate(test_data):
    img_num=data[1]
    img_data=data[0]
    orig=img_data
    data=img_data.reshape(50,50,1)
    model_out=model.predict([data])[0]
    if np.argmax(model_out)==1:
        str_label='Fracture'
    else:
        str_label='Not Fracture'
    plt.imshow(orig,cmap='gray')
    plt.title(str_label)
    plt.gca().set_axis_off()
    plt.show()
    break;


model.save('my_model.tflearn')

D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
